In [1]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
# from pandas_ml import ConfusionMatrix

In [2]:
with open("../input/kdd/kddcup.names",'r') as f:
    content = f.read()
    print(content)

back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster.
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.
diff_srv_rate: continuous.
srv_diff_host_rate: continuous.
dst_host_count: continuous.
dst_host_srv_count: con

In [3]:
cols = []
content = content.split("\n")[1:]
for c in content:
    if len(c) > 1:
        print(c.split(":")[0])
        cols.append(c.split(":")[0])
cols.append("target")

duration
protocol_type
service
flag
src_bytes
dst_bytes
land
wrong_fragment
urgent
hot
num_failed_logins
logged_in
num_compromised
root_shell
su_attempted
num_root
num_file_creations
num_shells
num_access_files
num_outbound_cmds
is_host_login
is_guest_login
count
srv_count
serror_rate
srv_serror_rate
rerror_rate
srv_rerror_rate
same_srv_rate
diff_srv_rate
srv_diff_host_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_serror_rate
dst_host_srv_serror_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate


In [4]:
with open("../input/kdd/training_attack_types",'r') as f:
    content = f.read()
    print(content)

back dos
buffer_overflow u2r
ftp_write r2l
guess_passwd r2l
imap r2l
ipsweep probe
land dos
loadmodule u2r
multihop r2l
neptune dos
nmap probe
perl u2r
phf r2l
pod dos
portsweep probe
rootkit u2r
satan probe
smurf dos
spy r2l
teardrop dos
warezclient r2l
warezmaster r2l




In [5]:
attack_types = dict()
attack_types["normal"] = "normal"
for line in content.split("\n"):
    if line:
        data = line.split(" ")
        attack_types[data[0]] = data[1]
print(attack_types)

{'normal': 'normal', 'back': 'dos', 'buffer_overflow': 'u2r', 'ftp_write': 'r2l', 'guess_passwd': 'r2l', 'imap': 'r2l', 'ipsweep': 'probe', 'land': 'dos', 'loadmodule': 'u2r', 'multihop': 'r2l', 'neptune': 'dos', 'nmap': 'probe', 'perl': 'u2r', 'phf': 'r2l', 'pod': 'dos', 'portsweep': 'probe', 'rootkit': 'u2r', 'satan': 'probe', 'smurf': 'dos', 'spy': 'r2l', 'teardrop': 'dos', 'warezclient': 'r2l', 'warezmaster': 'r2l'}


In [6]:
traindata = pd.read_csv('../kddcuptrain', names=cols)
testdata = pd.read_csv('../kddcuptrain', names=cols)

In [7]:
testdata.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [8]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494021 entries, 0 to 494020
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494021 non-null  int64  
 1   protocol_type                494021 non-null  object 
 2   service                      494021 non-null  object 
 3   flag                         494021 non-null  object 
 4   src_bytes                    494021 non-null  int64  
 5   dst_bytes                    494021 non-null  int64  
 6   land                         494021 non-null  int64  
 7   wrong_fragment               494021 non-null  int64  
 8   urgent                       494021 non-null  int64  
 9   hot                          494021 non-null  int64  
 10  num_failed_logins            494021 non-null  int64  
 11  logged_in                    494021 non-null  int64  
 12  num_compromised              494021 non-null  int64  
 13 

In [9]:
from sklearn.preprocessing import LabelEncoder


In [10]:
train = traindata.select_dtypes(exclude=[np.number])
print(train.columns)
for feature in train.columns:
    traindata[feature] = LabelEncoder().fit_transform(traindata[feature])

Index(['protocol_type', 'service', 'flag', 'target'], dtype='object')


In [11]:
test = testdata.select_dtypes(exclude=[np.number])
print(test.columns)
for feature in test.columns:
    testdata[feature] = LabelEncoder().fit_transform(testdata[feature])

Index(['protocol_type', 'service', 'flag', 'target'], dtype='object')


In [12]:
trainD = traindata.groupby('target', group_keys=False).apply(lambda x: x.sample(frac=0.05))


In [13]:
testD = testdata.groupby('protocol_type', group_keys=False).apply(lambda x: x.sample(frac=0.07))


In [14]:
trainD.shape

(24700, 42)

In [15]:
X = trainD.iloc[:,0:41]
Y = trainD.iloc[:,41]
C = testD.iloc[:,41]
T = testD.iloc[:,0:41]

In [16]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

In [17]:
scaler = Normalizer().fit(T)
testT = scaler.transform(T)

In [18]:
traindata = np.array(trainX)
trainlabel = np.array(Y)

In [19]:
testdata = np.array(testT)
testlabel = np.array(C)

In [20]:
import os

os.mkdir('resultM')
os.chmod('resultM', 0o777)  # gives read/write/execute permission to all users


In [21]:
model = LogisticRegression()
model.fit(traindata, trainlabel)

LogisticRegression()

In [22]:
# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('resultM/predictedLR.txt', predicted, fmt='%01d')


In [23]:
# summarize the fit of the model

cm = confusion_matrix(expected, predicted)
# print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print(accuracy_score(expected, predicted))
print("precision")
print(precision_score(expected, predicted ,average='weighted',  zero_division=1))
print("recall")
print(recall_score(expected, predicted ,average='weighted',  zero_division=1))
print("f-score")
print(f1_score(expected, predicted , average='weighted',  zero_division=1))
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)

0.000
0.000
Accuracy
0.9814354288358106
precision
0.978380081752967
recall
0.9814354288358106
f-score
0.9735533393712947
fpr
0.000
tpr
0.000


In [24]:
# fit a Naive Bayes model to the data
model = GaussianNB()
model.fit(traindata, trainlabel)
print(model)

GaussianNB()


In [25]:
# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('resultM/predictedNB.txt', predicted, fmt='%01d')


In [26]:
# summarize the fit of the model

cm = confusion_matrix(expected, predicted)
# print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print(accuracy_score(expected, predicted))
print("precision")
print(precision_score(expected, predicted ,average='weighted',  zero_division=1))
print("recall")
print(recall_score(expected, predicted ,average='weighted',  zero_division=1))
print("f-score")
print(f1_score(expected, predicted , average='weighted',  zero_division=1))
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)

0.979
1.000
Accuracy
0.868226244867272
precision
0.9925530165755334
recall
0.868226244867272
f-score
0.8987672151055172
fpr
1.000
tpr
0.979


In [27]:
# fit a k-nearest neighbor model to the data
model = KNeighborsClassifier()
model.fit(traindata, trainlabel)
print(model)
# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('resultM/predictedKNN.txt', predicted, fmt='%01d')


KNeighborsClassifier()


/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [28]:
# summarize the fit of the model

cm = confusion_matrix(expected, predicted)
# print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print(accuracy_score(expected, predicted))
print("precision")
print(precision_score(expected, predicted ,average='weighted',  zero_division=1))
print("recall")
print(recall_score(expected, predicted ,average='weighted',  zero_division=1))
print("f-score")
print(f1_score(expected, predicted , average='weighted',  zero_division=1))
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)

0.979
nan
Accuracy
0.9962119021456249
precision
0.9962762685454147
recall
0.9962119021456249
f-score
0.9959904400244024
fpr
nan
tpr
0.979


/var/folders/7h/btw3443n6hx6nbd24dh7dpjc0000gn/T/ipykernel_41303/3554530302.py:6: RuntimeWarning: invalid value encountered in true_divide
  fpr = float(cm[1][1])/np.sum(cm[1])


In [29]:
model = DecisionTreeClassifier()
model.fit(traindata, trainlabel)
print(model)
# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('resultM/predictedDT.txt', predicted, fmt='%01d')

DecisionTreeClassifier()


In [30]:
# summarize the fit of the model

cm = confusion_matrix(expected, predicted)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])

print("Accuracy")
print(accuracy_score(expected, predicted))
print("precision")
print(precision_score(expected, predicted , pos_label='positive',average='micro'))
print("recall")
print(recall_score(expected, predicted , pos_label='positive',average='micro'))
print("f-score")
print(f1_score(expected, predicted , pos_label='positive',average='micro'))
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)

Accuracy
0.9976288242438263
precision
0.9976288242438263
recall
0.9976288242438263
f-score
0.9976288242438263
fpr
nan
tpr
0.986


/var/folders/7h/btw3443n6hx6nbd24dh7dpjc0000gn/T/ipykernel_41303/1965532674.py:5: RuntimeWarning: invalid value encountered in true_divide
  fpr = float(cm[1][1])/np.sum(cm[1])
/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may us

In [31]:
model = AdaBoostClassifier(n_estimators=100)
model.fit(traindata, trainlabel)

AdaBoostClassifier(n_estimators=100)

In [32]:
# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('resultM/predictedABoost.txt', predicted, fmt='%01d')

In [33]:
# summarize the fit of the model

cm = confusion_matrix(expected, predicted)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])

print("Accuracy")
print(accuracy_score(expected, predicted))
print("precision")
print(precision_score(expected, predicted , pos_label='positive',average='micro'))
print("recall")
print(recall_score(expected, predicted , pos_label='positive',average='micro'))
print("f-score")
print(f1_score(expected, predicted , pos_label='positive',average='micro'))
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)

Accuracy
0.7855242611763346
precision
0.7855242611763346
recall
0.7855242611763346
f-score
0.7855242611763346
fpr
1.000
tpr
0.000


/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [34]:
model = RandomForestClassifier(n_estimators=100)
model.fit(traindata, trainlabel)

# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('resultM/predictedRF.txt', predicted, fmt='%01d')

In [35]:
predicted

array([18, 18, 18, ..., 11, 11, 11])

In [36]:
# summarize the fit of the model

cm = confusion_matrix(expected, predicted)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])

print("Accuracy")
print(accuracy_score(expected, predicted))
print("precision")
print(precision_score(expected, predicted , pos_label='positive',average='micro'))
print("recall")
print(recall_score(expected, predicted , pos_label='positive',average='micro'))
print("f-score")
print(f1_score(expected, predicted , pos_label='positive',average='micro'))
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)

Accuracy
0.998901162454456
precision
0.998901162454456
recall
0.998901162454456
f-score
0.998901162454456
fpr
1.000
tpr
0.993


/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/Users/praveenvenkatachalam/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
